## Add project root

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]  # This is /Users/atanugiri/Downloads/DLC-Jupyter-Notebooks
print(project_root)
sys.path.append(str(project_root))


## Connect to db

In [ ]:
import psycopg2
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.142"
conn = psycopg2.connect(
    dbname="deeplabcut_db", user="postgres", 
    password="1234", host=host, port="5432")
cursor = conn.cursor()


## Fetch id list

In [ ]:
import pandas as pd

task = 'ToyLight'

saline_q = f"""
SELECT id FROM dlc_table
WHERE task = '{task}' AND health = 'saline' AND head_x IS NOT NULL
ORDER BY id;
"""
df = pd.read_sql_query(saline_q, conn)
saline_id = df['id'].to_list()

ghrelin_q = f"""
SELECT id FROM dlc_table
WHERE task = '{task}' AND health = 'ghrelin' AND head_x IS NOT NULL
ORDER BY id;
"""
df = pd.read_sql_query(ghrelin_q, conn)
ghrelin_id = df['id'].to_list()

print(type(saline_id))


## Calculate coccupancy per grid

In [ ]:
import importlib
import Python_scripts.Feature_functions.occupancy_grid

importlib.reload(Python_scripts.Feature_functions.occupancy_grid)

from Python_scripts.Feature_functions.occupancy_grid import (
    compute_occupancy_grid, get_batch_occupancy_grids
)


### Export occupancy probablities

In [ ]:
import numpy as np

for grid_size in range(3,10):
    g_saline = get_batch_occupancy_grids(saline_id, conn, grid_size=grid_size)
    np.save(f'{task}_saline_{grid_size}.npy', g_saline)

for grid_size in range(3,10):
    g_ghrelin = get_batch_occupancy_grids(ghrelin_id, conn, grid_size=grid_size)
    np.save(f'{task}_ghrelin_{grid_size}.npy', g_ghrelin)    


In [ ]:
# # Extract only the grid from (id, grid) tuples
# g_saline_grids = [grid for _, grid in g_saline]
# g_ghrelin_grids = [grid for _, grid in g_ghrelin]

# # Flatten
# g_saline_stack = np.stack(g_saline_grids)          # shape: (num_trials, 10, 10)
# g_saline_avg = np.mean(g_saline_stack, axis=0)     # shape: (10, 10)
# bin_probs_saline = g_saline_avg.flatten()          # shape: (100, )

# print(g_saline_grids[0])
# print(g_saline_stack.shape)
# print(g_saline_avg.shape)
# print(bin_probs_saline.shape)

# g_ghrelin_stack = np.stack(g_ghrelin_grids)
# g_ghrelin_avg = np.mean(g_ghrelin_stack, axis=0)
# bin_probs_ghrelin = g_ghrelin_avg.flatten()

# r_saline = g_saline_stack.flatten()   # shape: (n_trials * grid_size * grid_size,)
# r_ghrelin = g_ghrelin_stack.flatten()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.bar(np.arange(100), bin_probs_saline, label='Saline', alpha=0.6)
plt.bar(np.arange(100), bin_probs_ghrelin, label='Ghrelin', alpha=0.6)
plt.xlabel('Bin Index (row-wise)')
plt.ylabel('Probability of being in bin')
plt.title('Spatial Occupancy per Bin (Group Average)')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.imshow(g_saline_avg, cmap='hot', origin='lower')
plt.colorbar(label='Probability')
plt.title('Saline Occupancy Map')

plt.subplot(1, 2, 2)
plt.imshow(g_ghrelin_avg, cmap='hot', origin='lower')
plt.colorbar(label='Probability')
plt.title('Ghrelin Occupancy Map')

plt.tight_layout()
plt.show()
